In [1]:
pip install PyPDF2

Note: you may need to restart the kernel to use updated packages.


In [1]:
import PyPDF2

### Testing

In [2]:
#Reading the PDF

f = open('data/october_permits.pdf','rb')

In [3]:
#Extracting all of the text from the PDF
pdf_reader = PyPDF2.PdfFileReader(f) 

#Outputting the number of pages
pdf_reader.numPages

12

In [4]:
#Getting the text from a specific page
#Page one would be getPage(0) and page 2 would be getPage(1) and so on

#Here I will get all of the text from page 4

page_four = pdf_reader.getPage(3)
page_four_text = page_four.extractText()

page_four_text

'PERMITS ISSUED BY WORK CLASS (10/01/2022 TO 10/31/2022)\nPermit # Main Address Type Workclass District Status Parcel\nApplication Date Issue Date Expiration Last Inspection Finaled DateGeneral Contractor\nFee Total Valuation Sq Ft Zone Assigned To\nPERMITS ISSUED FOR ADDITION (RESIDENTIAL):   6 $457,351.44 \nALTERATION (COMMERCIAL)\nBSD-Building \n(Commercial)Alteration \n(Commercial)5400 Paris Rd, Col, MO 65202 Third Issued 1250300100010001 BLDC-004637-2022\n10/11/2022 04/03/2023 10/05/2022 09/08/2022\nWes Davis $2,295.00 $680,000.00  8,700 A AgriculturalSeptagon Construction Company, Inc.-Columbia\nPermits Septagon\nDescription:   The 3M Columbia, MO plant is planning to remove the existing Washline 1 and install a new dynamic washline and glycerin dryer oven in the Kahuna process area. The \nproject will plan space and utilities for up to four total dynamic washlines in the future. This project includes modifications to the existing mechanical HVAC and piping in the area, \nmodific

In [5]:
#Now that we have all of this in text format, we can use regular expressions to get the info we need
import re

In [90]:
permits = re.findall(r'(BLDC-\d{6}-\d{4})',page_four_text)

#We need to take into account the different prefixes that can be in these permit codes. 
#There is BLDC, BLDR, POOL, DECK, ROOF and DEMO

permits = re.findall(r'((?:BLDC|BLDR|POOL|DECK|ROOF|DEMO)-\d{6}-\d{4})',page_four_text)
permits


['BLDC-004637-2022',
 'BLDC-004186-2022',
 'BLDC-000201-2023',
 'BLDC-004572-2022',
 'BLDC-004698-2022',
 'BLDC-004698-2022']

In [97]:
#Now for the permit types

types = re.findall(r'BSD-Building \n([(]\S*)',page_four_text)

#We need to take into account the different prefixes that can be in these permit codes. 
#There is BSD-Building, BSD-Pool/Spa, BSD-Deck Only, BSD-Reroof, BSD-Demolition

types = re.findall(r'(?:BSD-Building|BSD-Pool/Spa|BSD-Deck Only|BSD-Reroof|BSD-Demolition) \n([(]\S*)',page_four_text)
types

['(Commercial)Alteration',
 '(Commercial)Alteration',
 '(Commercial)Alteration',
 '(Commercial)Alteration',
 '(Commercial)Alteration',
 '(Commercial)Alteration']

In [9]:
#Application Date
application_date = re.findall(r'(\d{2}\/\d{2}\/\d{4})\n\D',page_four_text)
application_date

['09/08/2022',
 '08/10/2022',
 '10/10/2022',
 '09/01/2022',
 '09/13/2022',
 '09/13/2022']

In [79]:
#Issued Date
issued_date = re.findall(r'(\d{2}\/\d{2}\/\d{4})\s\d{2}\/\d{2}\/\d{4}\n\D',page_four_text)
issued_date

['10/05/2022',
 '10/06/2022',
 '10/10/2022',
 '10/11/2022',
 '10/12/2022',
 '10/12/2022']

In [ ]:
#The expiration and last inspection dates are harder to get because sometimes dates are missing, and that makes it 
#more difficult to write a regular expression

In [71]:
#Description
description = re.findall(r'Description:   (.*?)~ BSD-Building', page_four_text.replace('\n','~ '))
description

['The 3M Columbia, MO plant is planning to remove the existing Washline 1 and install a new dynamic washline and glycerin dryer oven in the Kahuna process area. The ~ project will plan space and utilities for up to four total dynamic washlines in the future. This project includes modifications to the existing mechanical HVAC and piping in the area, ~ modifications to lighting and power distribution, and architectural and structural modifications.',
 'Interior alteration for mental health counseling center for veterans. The scope of work is limited to the lower level of the existing building.',
 'Non structural repairs to exterior wall from vehicle impact.',
 'Adding walls to make offices, ceilings and lights.   Has 2 existing HC baths - no work.   Has water & electric meters.  Non-sprinklered.   Minor demo of a few walls & ~ ceilings. Type 2B const. Offices.',
 'Remodel and Related Improvements to "The Wylder" Apartment Complex.~ 10 apartment buildings.~ Level 2 Alteration of all 172 a

In [78]:
#Address
address = re.findall(r'[)](\d.*?\d{5})', page_four_text.replace('\n',''))
address


['5400 Paris Rd, Col, MO 65202',
 '2475 Broadway Bluffs Dr, STE 1, Columbia, MO 65201',
 '1912 W Worley St, Columbia, MO 65203',
 '3400 Broadway Business Park Ct STE, 102, Col, MO 65203',
 '301 Campusview Dr, Col, MO 65201',
 '301 Campusview Dr, Col, MO 65201']

In [ ]:
#Note they all have addresses -- just have to get the full text of all of the pages so that you have it in order

In [38]:
#Square footage
square_feet = re.findall(r'\$\d*\s{2}(\d*)\s\D', page_four_text.replace(',','').replace('.',''))
square_feet


['8700', '5725', '0', '2391', '200000', '200000']

In [41]:
#Status of the permit
status = re.findall(r'(?:First|Second|Third|Fourth|Fifth|Sixth|Boone)\s(\D*)\s\d', page_four_text)
status

['Issued', 'Issued', 'Issued', 'Issued', 'Issued', 'Issued']

In [43]:
#Parcel
parcel = re.findall(r'(\d{16})', page_four_text)
parcel


['1250300100010001',
 '1740200170010001',
 '1622000030010101',
 '1650100020390901',
 '1660400020060001',
 '1660400020060001']

In [56]:
#Project cost

cost = re.findall(r'\$\d*[.]\d*\s\$(\d*[.]\d*)', page_four_text.replace(',',''))
cost

['680000.00', '800000.00', '0.00', '254989.10', '3936000.00', '3936000.00']

### Applying to the entire PDF

In [168]:
#Putting it all together

#First combine all of the pages to get one long string of text

f = open('data/october_permits.pdf','rb')
pdf_reader = PyPDF2.PdfFileReader(f) 

number_of_pages = pdf_reader.getNumPages()

text=[]

for x in range(number_of_pages):  
    page = pdf_reader.getPage(x)
    page_content = page.extractText()
    text.append(page_content)
    
#Getting text as a string

full_text = ' '.join(text)
full_text


'PERMITS ISSUED BY WORK CLASS (10/01/2022 TO 10/31/2022)\nFOR CITY OF COLUMBIA, MISSOURI\n0M 2M 4M 6M 8M 10MADDITION (RESIDENTIAL)\nALTERATION (COMMERCIAL)\nALTERATION (RESIDENTIAL)\nBSD - POOL (1&2 FAMILY)\nCOMMERCIAL MULTI-FAMILY\nCOMMERCIAL NEW\nDECK ONLY\nDUPLEX\nRE-ROOF\nRESIDENTIAL DEMO\nSINGLE FAMILY DETACHED457,351\n9,621,989\n298,567\n28,000\n7,805,310\n311,917\n81,355\n1,264,313\n1,924,112\n0\n5,802,472Valuation 0 2 4 6 8 10 12 14ADDITION (RESIDENTIAL)\nALTERATION (COMMERCIAL)\nALTERATION (RESIDENTIAL)\nBSD - POOL (1&2 FAMILY)\nCOMMERCIAL MULTI-FAMILY\nCOMMERCIAL NEW\nDECK ONLY\nDUPLEX\nRE-ROOF\nRESIDENTIAL DEMO\nSINGLE FAMILY DETACHED6\n8\n9\n1\n1\n1\n5\n4\n13\n3\n11 PERMITS ISSUED BY WORK CLASS (10/01/2022 TO 10/31/2022)\nPermit # Main Address Type Workclass District Status Parcel\nApplication Date Issue Date Expiration Last Inspection Finaled DateGeneral Contractor\nFee Total Valuation Sq Ft Zone Assigned To\nADDITION (RESIDENTIAL)\nBSD-Building \n(Residential \nAlter/Add)

In [179]:
#Processing

permits = re.findall(r'((?:BLDC|BLDR|POOL|DECK|ROOF|DEMO)-\d{6}-\d{4})',full_text)
len(permits)

parcel = re.findall(r'(\d{16})|...Pending..', full_text)
len(parcel)
#This is tricky because not all have a parcel number

types = re.findall(r'(?:BSD-Building|BSD-Pool/Spa|BSD-Deck Only|BSD-Reroof|BSD-Demolition) (.*?)\d',full_text.replace('\n','|'))
len(types)

description = re.findall(r'Description:   (.*?)~ (?:BSD-Building|BSD-Pool/Spa|BSD-Deck Only|BSD-Reroof|BSD-Demolition|Permit|PERMITS|Page)|...Pending..', full_text.replace('\n','~ '))
len(description)

address = re.findall(r'(?:[)]|Multi-Family|Demo\s|Re-Roof\s|Pool\s|Detached|Duplex\s|New\s|Only\s)(\d.*?\d{5})', full_text.replace('\n',''))
len(address)

cost = re.findall(r'\$\d*[.]\d*\s\$(\d*[.]\d*)', full_text.replace(',',''))
len(cost)

application_date = re.findall(r'(\d{2}\/\d{2}\/\d{4})\n\D',full_text)
len(application_date)

issued_date = re.findall(r'(\d{2}\/\d{2}\/\d{4})\s\d{2}\/\d{2}\/\d{4}\n\D',full_text)
len(issued_date)

square_feet = re.findall(r'\$\d*\s\$\d*\s{2}(\d*)(?:\s\D|[A-Z])', full_text.replace(',','').replace('.',''))
len(square_feet)

status = re.findall(r'(?:First |Second |Third |Fourth |Fifth |Sixth |County|[.])(\D*)\s(?:\d{16}|POOL)', full_text)
len(status)


62

In [189]:
#Putting it together into a dataframe

import pandas as pd 

final_permits = pd.DataFrame(
    {'permitID': permits,
     'parcel': parcel,
     'type': types,
     'description': description,
     'address': address,
     'cost': cost,
     'application_date': application_date,
     'issued_date': issued_date,
     'sqft': square_feet,
     'status': status    
    })

In [194]:
final_permits.head()

,permitID,parcel,type,description,address,cost,application_date,issued_date,sqft,status
0,BLDR-000083-2023,1632000090120001,residential addition,12' x 40' detached wood framed storage buildin...,"600 Rogers St, Columbia, MO 65201",31910.40,10/05/2022,10/05/2022,480,Issued
1,BLDR-000390-2023,1631300030170001,residential addition,garage and storage addition within existing se...,"505 Pershing Rd, Col, MO 65203",79776.00,10/24/2022,10/26/2022,1200,Issued
2,BLDR-000399-2023,1680000060400001,residential addition,4 season room at rear of home.,"1923 Devonshire Dr, Col, MO 65203",77725.44,10/24/2022,10/26/2022,468,Issued
3,BLDR-000309-2023,1720400000250001,residential addition,31' x 31' pole barn style addition to existing...,"4810 St Charles Rd, Columbia, MO 65201",63887.28,10/17/2022,10/28/2022,961,Issued
4,BLDR-000454-2023,1760000011790001,residential addition,Add bonus room and additional garage,"904 Old Hawthorne Drive East, Columbia, MO 65201",100418.40,10/27/2022,10/28/2022,1100,Issued


In [195]:
#Some cleaning of the types column

final_permits['type'] = final_permits['type'].replace(['|(Residential |Alter/Add)Addition |(Residential)'], 'residential addition')
final_permits['type'] = final_permits['type'].replace(['|(Commercial)Alteration |(Commercial)'], 'commercial alteration')
final_permits['type'] = final_permits['type'].replace(['|(Residential |Alter/Add)Alteration |(Residential)'], 'residential alteration')
final_permits['type'] = final_permits['type'].replace(['BSD - Pool ('], 'pool (1&2 family)')
final_permits['type'] = final_permits['type'].replace(['|(Commercial)Commercial |Multi-Family'], 'commericial multifamily')
final_permits['type'] = final_permits['type'].replace(['|(Commercial)Commercial New '], 'commercial new')
final_permits['type'] = final_permits['type'].replace(['Deck Only '], 'deck')
final_permits['type'] = final_permits['type'].replace(['|(Residential New)Duplex '], 'new duplex')
final_permits['type'] = final_permits['type'].replace(['Re-Roof '], 're-roof')
final_permits['type'] = final_permits['type'].replace(['|Structure(s)Residential Demo '], 'residential demolition')
final_permits['type'] = final_permits['type'].replace(['|(Residential New)Single Family |Detached'], 'single family detached')


In [196]:
final_permits.type.unique()

array(['residential addition', 'commercial alteration',
       'residential alteration', 'pool (1&2 family)',
       'commericial multifamily', 'commercial new', 'deck', 'new duplex',
       're-roof', 'residential demolition', 'single family detached'],
      dtype=object)

In [202]:
#Cleaning the address -- converting Col to Columbia

final_permits.replace('Col,','Columbia,',regex=True,inplace=True)
final_permits.head()

,permitID,parcel,type,description,address,cost,application_date,issued_date,sqft,status
0,BLDR-000083-2023,1632000090120001,residential addition,12' x 40' detached wood framed storage buildin...,"600 Rogers St, Columbia, MO 65201",31910.40,10/05/2022,10/05/2022,480,Issued
1,BLDR-000390-2023,1631300030170001,residential addition,garage and storage addition within existing se...,"505 Pershing Rd, Columbia, MO 65203",79776.00,10/24/2022,10/26/2022,1200,Issued
2,BLDR-000399-2023,1680000060400001,residential addition,4 season room at rear of home.,"1923 Devonshire Dr, Columbia, MO 65203",77725.44,10/24/2022,10/26/2022,468,Issued
3,BLDR-000309-2023,1720400000250001,residential addition,31' x 31' pole barn style addition to existing...,"4810 St Charles Rd, Columbia, MO 65201",63887.28,10/17/2022,10/28/2022,961,Issued
4,BLDR-000454-2023,1760000011790001,residential addition,Add bonus room and additional garage,"904 Old Hawthorne Drive East, Columbia, MO 65201",100418.40,10/27/2022,10/28/2022,1100,Issued


In [204]:
#Removing the ~'s that I inserted at various points in the process to help with the regex

final_permits['description'].replace('~', '', regex=True, inplace=True)

In [206]:
#Writing the final result to a CSV
final_permits.to_csv('october_permits.csv', index=False)